In [0]:
from pathlib import Path
import networkx as nx
from networkx import NetworkXError
import requests
import pandas as pd
from random import random
from json import JSONDecodeError

ROOT = Path('drive/My Drive/PWr/ONoS/Task2')

In [0]:
def init_graph(network_name):
    g = nx.Graph()
    network_path = ROOT/'networks'/network_name
    with open(network_path / 'edges.csv', "r") as f:
        g = nx.parse_edgelist(f, nodetype=int, create_using=g)
    try:
        with open(network_path / 'nodes.csv', "r") as f:
            node_labels = map(lambda x: x.rstrip('\n'), f.readlines())
            mapping = {k: v for k, v in enumerate(node_labels, start=1)}
            g = nx.relabel_nodes(g, mapping)
    except IOError:
        print("Unlabeled graph")
    return g

### Geofinder
Looking for lattitudes nad longtitudes using **geocode.xyz** API and writing values in csv file. Because of very poor performance of **geocode.xyz**, not helpful in analyzing data related to geopostion.

In [0]:
g = init_graph('subelj_euroroad')
latts = [0] * len(g.nodes())
longts = [0] * len(g.nodes())

In [0]:
def read_from_csv(infile):
  global latts, longts
  df = pd.read_csv(infile)
  latts = list(df["latt"])
  longts = list(df["longt"])

read_from_csv(ROOT/'geodata.csv')

In [0]:
def write_to_csv(cities,latts,longts,outfile):
  with open(outfile,'w+') as f:
    d = {'city' : cities, 'latt' : latts, 'longt' : longts }
    df = pd.DataFrame(d)
    df.to_csv(f, encoding='utf-8')

cities = list(g.nodes())
write_to_csv(cities,latts,longts,ROOT/'geodata.csv')

In [0]:
def get_city_positions(cities):
  global latts, longts
  positionURL = lambda city : 'https://geocode.xyz/'+city+'?json=1'
  
  for idx, city in enumerate(cities):
    if longts[idx] == 0:
      try:
        res = requests.get(positionURL(city)).json()
        latts[idx] = float(res['latt'])
        longts[idx] = float(res['longt'])
      except (KeyError, JSONDecodeError):
        pass
      if idx > 10 and sum(longts[idx-10:idx]) == 0 : 
        break
      print(idx,longts[idx])

In [0]:
cities = list(g.nodes())
get_city_positions(cities)